A mesma base de dados que vem sendo trabalhada na disciplina também será usada aqui.

O problema agora é montar modelos para classificar postagens entre engajamento alto e aquelas que não geraram engajamento alto.
Define-se engajamento alto como as postagens com engajament_total entre os 25% maiores valores (como no Trabalho 4).
Aqui, as variáveis que foram usadas para calcular engajament_total não devem ser usadas como atributos para o modelo.

Vocês devem desenvolver modelos para ao menos 2 tipos de classificadores, sendo que ao menos um deles deve possibilitar extrair algum conhecimento; apresente isso (atributos importantes ou regras condicionais, por exemplo).
Deve-se:
- melhorar o tratamento dos dados, se necessário;
- adotar uma estratégia experimental adequada;
- avaliar comparativamente os modelos, incluindo análises de hiper-parâmetros;
- tratar o desbalanceamento dos dados; e
- extrair conhecimento de ao menos um dos modelos.

O conteúdo a ser enviado é:
(1) apresentação do desenvolvimento do trabalho e resultados alcançados (não mais que 15min),
(2) códigos (notebooks em arquivo ipynb, se preferirem) usados na resolução do trabalho.
Os códigos são apenas para consulta/dúvidas e não fazem parte da avaliação. Como comentado antes, a apresentação pode ser feita via slides ou num notebook (por exemplo, do Google Colab) e pode conter os códigos usados no trabalho. Neste caso, não precisa enviar os códigos num arquivo separado. Vale destacar que não é obrigatório usar Python e Google Colab.

In [1]:
import math
from functools import reduce
from utils import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from utils.utils import *

In [2]:
df = pd.read_csv('preprocessing_result1.csv')
df

,content_type,tone,emotion_displayed,location_type,followers,likes,num_comments,video_view_count,has_a_person,vape_present,...,post_style_array_Promotion,type_of_content_array_Informational,type_of_content_array_Advertisement,type_of_content_array_Promotion,type_of_content_array_Warning,type_of_content_array_Campaign,type_of_content_array_Health,target_demographic_Other,type_of_content_Other,post_style_Other
0,Image,Negative,other,Public,0.666791,0.09375,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,Image,Negative,other,Public,0.058824,0.06250,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
2,Image,Negative,other,Indoor,1.000000,0.37500,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,Image,Sarcastic,annoyance,Indoor,0.002978,0.09375,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
4,Image,Positive,other,Private,0.118019,0.21875,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
978,Image,Negative,other,Public,0.067759,0.12500,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
979,Image,Positive,happiness,Public,0.970588,0.25000,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
980,Image,Negative,other,Public,0.003723,0.00000,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
981,Image,Positive,happiness,Outdoor,0.193596,1.00000,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
